<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/FileVital.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [112]:
import pandas as pd
hemo = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Hemodialysis.csv', encoding='latin1')
hemo.shape

(149987, 5)

In [113]:
import pandas as pd
vital = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/vitalsign.csv')
vital.shape

(9213497, 7)

In [114]:
# Create a list from hemo['PatientUID']
patient_uid_list = hemo['PatientUID'].unique().tolist()

# Remove patient UIDs from visit['PatientUID'] that are not in the list
vital = vital[~vital['PatientUID'].isin(patient_uid_list)]
vital.shape

(8099274, 7)

In [115]:
# Specify the columns for unique value counting
columns_to_count = ['OrderItemName', 'ResultItemName']

# Count unique values in the specified columns
unique_counts = vital[columns_to_count].value_counts().reset_index()

In [116]:
unique_counts.columns = ['OrderItemName','ResultItemName', 'Count']

In [117]:
selected_items = ['Systolic BP', 'Diastolic BP', 'BMI', 'BSA']
filtered_items = unique_counts[unique_counts['ResultItemName'].isin(selected_items)]
filtered_items

,OrderItemName,ResultItemName,Count
0,Vital Signs Record,Systolic BP,1221173
1,Vital Signs Record,Diastolic BP,1220095
7,Vital Signs Record,BMI,558791
8,Vital Signs Record,BSA,558502


In [118]:
#filtered_items.to_excel('VitalFile.xlsx')

In [119]:
import pandas as pd
slope = pd.read_excel('/content/drive/MyDrive/Thesis Project Kidney/Assign/Assign 3: Re-CountSlope & Quartile/LabSlope.xlsx')

In [120]:
data = pd.DataFrame(slope)
data = data[['PatientUID','PatientVisitUID','Slopes']]
data = data.dropna()
data

,PatientUID,PatientVisitUID,Slopes
1,2299,4020970,0.002857
2,2299,4286612,0.020833
3,2299,4546471,-0.062921
4,2299,4801938,-0.056522
5,2299,5051781,0.047619
...,...,...,...
280094,3409733,10435209,-0.208313
280095,3432056,10235244,0.168400
280096,3432056,10306813,1.129412
280097,3432056,10408954,0.035000


## **TestWideTable**


In [142]:

# Initialize a dictionary to store the vital values
vital_dict = {}

# Iterate through unique ResultItemName values
for result_item in vital['ResultItemName'].unique():
    # Filter vital DataFrame for the current result_item
    subset_vital = vital[vital['ResultItemName'] == result_item]

    # Merge the subset_vital with data on PatientUID and PatientVisitUID
    merged_data = pd.merge(data, subset_vital[['PatientUID', 'PatientVisitUID', 'ResultValue', 'CWhen']], on=['PatientUID', 'PatientVisitUID'], how='left')

    # Rename the ResultValue column to match the result_item
    merged_data.rename(columns={'ResultValue': result_item}, inplace=True)

    # Update the vital_dict with the result_item as the key and merged_data as the value
    vital_dict[result_item] = merged_data

# Concatenate the dictionary values to get the final result
final_result = pd.concat(vital_dict.values(), axis=1)

# Drop duplicate columns (except for 'PatientUID', 'PatientVisitUID', and 'Slopes')
final_result = final_result.loc[:, ~final_result.columns.duplicated()]


### **solved the dup rows**


In [143]:

# Sort data by 'CWhen' in descending order within each patient's visit and keep the latest visit row
final_result.sort_values(by=['PatientUID', 'PatientVisitUID', 'CWhen'], ascending=[True, True, False], inplace=True)
final_result.drop_duplicates(subset=['PatientUID', 'PatientVisitUID'], keep='first', inplace=True)


In [144]:
vitalDF = final_result[['PatientUID','PatientVisitUID','Slopes','Systolic BP', 'Diastolic BP', 'BMI', 'BSA']]
# Drop rows with missing values in 'PatientUID', 'PatientVisitUID', and 'Slopes'
vitalDF = vitalDF.dropna(subset=['PatientUID', 'PatientVisitUID', 'Slopes'])

In [145]:
vitalDF.shape

(280098, 7)

In [146]:
data.shape

(280098, 3)

In [147]:
vitalDF

,PatientUID,PatientVisitUID,Slopes,Systolic BP,Diastolic BP,BMI,BSA
0,2299.0,4020970.0,0.002857,127,79,28.62,1.6
2,2299.0,4286612.0,0.020833,129,82,21.94,1.41
3,2299.0,4546471.0,-0.062921,146,81,20.27,1.38
4,2299.0,4801938.0,-0.056522,105,64,NaN,NaN
5,2299.0,5051781.0,0.047619,111,58,20.49,1.38
...,...,...,...,...,...,...,...
289491,3409733.0,10435209.0,-0.208313,155,77,NaN,NaN
289492,3432056.0,10235244.0,0.168400,114,73,NaN,NaN
289493,3432056.0,10306813.0,1.129412,142,69,NaN,NaN
289494,3432056.0,10408954.0,0.035000,130,77,NaN,NaN


## **Check Duplicate vitalDF**
    cuz there are the same visitID but in different Lab
    

In [148]:
# Check for duplicate rows in data
duplicates_data = data[data.duplicated(['PatientUID', 'PatientVisitUID'], keep=False)]

# Check for duplicate rows in vitalDF
duplicates_vitalDF = vitalDF[vitalDF.duplicated(['PatientUID', 'PatientVisitUID', 'Slopes'], keep=False)]


In [149]:
duplicates_vitalDF

,PatientUID,PatientVisitUID,Slopes,Systolic BP,Diastolic BP,BMI,BSA


    this is visitID that had dup value
    but we select only latest value

In [152]:
vital[vital['PatientVisitUID']==4286612]

,LabNo,PatientUID,PatientVisitUID,OrderItemName,ResultItemName,ResultValue,CWhen
338602,1,2299,4286612,Vital Signs Record,BSA,1.42,2560-07-05 09:00:18.093
338603,2,2299,4286612,Vital Signs Record,BSA,1.41,2560-07-05 14:29:33.277
338604,1,2299,4286612,Vital Signs Record,Systolic BP,138,2560-07-05 09:00:18.093
338605,2,2299,4286612,Vital Signs Record,Systolic BP,129,2560-07-05 14:29:33.277
338606,1,2299,4286612,Vital Signs Record,Waistline,72,2560-07-05 14:29:33.277
338607,1,2299,4286612,Vital Signs Record,Weight,48.6,2560-07-05 09:00:18.093
338608,2,2299,4286612,Vital Signs Record,Weight,48.7,2560-07-05 14:29:33.277
338753,1,2299,4286612,Vital Signs Record,BMI,21.6,2560-07-05 09:00:18.093
338754,2,2299,4286612,Vital Signs Record,BMI,21.94,2560-07-05 14:29:33.277
338755,1,2299,4286612,Vital Signs Record,Pulse Rate,76,2560-07-05 09:00:18.093


In [154]:
vitalDF[vitalDF['PatientVisitUID']==4286612.0]

,PatientUID,PatientVisitUID,Slopes,Systolic BP,Diastolic BP,BMI,BSA
2,2299.0,4286612.0,0.020833,129,82,21.94,1.41


In [155]:
vitalDF.to_excel('vitalDF.xlsx')